# Advent of Code 2018
*Phong Nguyen, December 2018*

## Lessons Learnt


### Some utility functions

In [65]:
import math
import re
from collections import deque, defaultdict, Counter
from itertools import chain, combinations
from heapq import heappop, heappush

flatten = chain.from_iterable

from numba import jit

def Input(day):
    "Return input file."
    return open('input{}.txt'.format(day))

def InputString(day):
    "Return the content of the input file as a string."
    return Input(day).read()

def InputRows(day):
    "Return the content of the input file as a list of string, each for a row."
    return InputString(day).splitlines()

def InputInts(day):
    "Return the content of the input file as a list of integers, each for a row."
    return [int(x) for x in InputString(day).splitlines()]

def ints(start, end, step=1): return range(start, end+1, step)

# 2D points
UP, LEFT, DOWN, RIGHT = (0, -1), (-1, 0), (0, 1), (1, 0)

def add_tuples(t1, t2):
     return tuple(sum(x) for x in zip(t1, t2))
        
def Mht_distance(p):
    return abs(p[0]) + abs(p[1])

def argmax(a):
    return a.index(max(a))

def argmax_dict(d):
    return max(d, key=(lambda k: d[k]))

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def breadth_first(start, goal, moves_func):
    "Find a shortest sequence of states from start to the goal."
    frontier = deque([start]) # A queue of states
    previous = {start: None}  # start has no previous state; other states will
    while frontier:
        s = frontier.popleft()
        if s == goal:
            return path(previous, s)
        for s2 in moves_func(s):
            if s2 not in previous:
                frontier.append(s2)
                previous[s2] = s
                
def path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return [] if (s is None) else path(previous, previous[s]) + [s]

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.

    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
                
    return dict(fail=True, front=len(frontier), prev=len(previous))

## Day 1: Chronal Calibration
[Problem Description](https://adventofcode.com/2018/day/1)

In [2]:
def day1a(nums):
    return sum(nums)

nums = InputInts(1)
day1a(nums)

505

In [3]:
def day1b(nums):
    seen = set()
    f = 0
    while True:
        for x in nums:
            f += x
            if f in seen:
                return f
            else:
                seen.add(f)
        
day1b(nums)

72330

## Day 2: Inventory Management System
[Problem Description](https://adventofcode.com/2018/day/2)

In [4]:
def has_two(text):
    return 2 in Counter(text).values()

def has_three(text):
    return 3 in Counter(text).values()
    
def day2a(lines):
    two = sum(1 for l in lines if has_two(l))
    three = sum(1 for l in lines if has_three(l))
    return two * three

lines = InputRows(2)
day2a(lines)

8892

In [5]:
def are_correct(t1, t2):
    diff_count = 0
    for i in range(len(t1)):
        if t1[i] != t2[i]:
            diff_count += 1
        if diff_count > 1:
            return False
    return diff_count == 1
    
def day2b(lines):
    for c in combinations(lines, 2):
        if are_correct(*c):
            t1, t2 = c
            common_string = ''
            for i in range(len(t1)): 
                if t1[i] == t2[i]: 
                    common_string += t1[i]
            print(common_string)
                
%time day2b(lines)

zihwtxagifpbsnwleydukjmqv
CPU times: user 38.8 ms, sys: 2.72 ms, total: 41.5 ms
Wall time: 39.1 ms


A shorter way (less code) and also easier to find the common string for the answer is to remove each character and check if the rest of the two strings are the same. It is slower though.

In [6]:
%%time

for c in combinations(lines, 2):
    t1, t2 = c
    for i in range(len(t1)):
        x1 = t1[:i] + t1[i+1:]
        x2 = t2[:i] + t2[i+1:]
        if x1 == x2:
            print(x1)
            break

zihwtxagifpbsnwleydukjmqv
CPU times: user 646 ms, sys: 8.33 ms, total: 654 ms
Wall time: 649 ms


Reading solutions in reddit, I see using `zip` is a neat way of looping through pairs of things. Also, a more pythonic way of summing booleans. So, rewrite my first solution.

In [7]:
def are_correct_zip(t1, t2):
    return sum(c1 != c2 for c1, c2 in zip(t1, t2)) == 1
    
def day2b_zip(lines):
    for c in combinations(lines, 2):
        if are_correct(*c):
            common_string = ''.join(c1 for c1, c2 in zip(*c) if c1 == c2)
            print(common_string)
                
%time day2b_zip(lines)

zihwtxagifpbsnwleydukjmqv
CPU times: user 39 ms, sys: 1.86 ms, total: 40.9 ms
Wall time: 39.6 ms


Well, it is as fast as the original one, even though it needs to sum up all differences, without early exit.

## Day 3: No Matter How You Slice It
[Problem Description](https://adventofcode.com/2018/day/3)

I started with a bad algorithm, checking each pixel against all claims. There are 1 millions pixel, so too slow. Then, I switch to a more active approach: recording the area of each claim and check how many pixels are recorded more than twice. As the size of a claim is small, this approach is much faster.

In [8]:
def parse_claim(line):
    "Return x, y, w, h"
    nums = re.findall(r'\d+', line)[1:]
    return tuple(map(int, nums))

def day3(lines):
    claims = list(map(parse_claim, lines))
    lookup = defaultdict(int)

    for c in claims:
        for x in range(c[0], c[0] + c[2]):
            for y in range(c[1], c[1] + c[3]):
                lookup[(x, y)] += 1
    
    print('part 1', sum(c > 1 for c in lookup.values()))
    
    def is_overlapped(c):
        for x in range(c[0], c[0] + c[2]):
            for y in range(c[1], c[1] + c[3]):
                if lookup[(x, y)] > 1:
                    return True

    for i, c in enumerate(claims):
        if not is_overlapped(c):
            print('part 2', i + 1)

day3(InputRows(3))

part 1 115304
part 2 275


## Day 4: Repose Record
[Problem Description](https://adventofcode.com/2018/day/4)

In [60]:
def day4(rows):
    rows = sorted(rows)
    records = []
    total_minutes_lookup = defaultdict(int)
    minute_lookup = defaultdict(Counter)
    start = None
    
    for row in rows:
        nums = list(map(int, re.findall(r'\d+', row)))
        if len(nums) == 6:
            guard = nums[5]
        else:
            month, day, hour, minute = nums[1:5]
            if start == None:
                start = minute
            else:
                stop = minute
                total_minutes_lookup[guard] += (stop - start)
                minute_lookup[guard].update(range(start, stop))
                start = None
        
    most_sleep_guard = argmax_dict(total_minutes_lookup)
    most_minute = minute_lookup[most_sleep_guard].most_common(1)[0][0]
    print('part 1', most_sleep_guard * most_minute)

    max_all = [(guard, *minute_lookup[guard].most_common(1)[0]) for guard in minute_lookup]
    guard, minute, _ = max(max_all, key=(lambda a:a[2]))
    print('part 2', guard * minute)

day4(InputRows(4))

part 1 84636
part 2 91679


In [10]:
%load_ext autoreload
%autoreload 2

## Day 5: Repose Record
[Problem Description](https://adventofcode.com/2018/day/5)

In [84]:
p = re.compile(r'aA|Aa|bB|Bb|cC|Cc|dD|Dd|eE|Ee|fF|Ff|gG|Gg|hH|Hh|iI|Ii|jJ|Jj|kK|Kk|lL|Ll|mM|Mm|nN|Nn|oO|Oo|pP|Pp|qQ|Qq|rR|Rr|sS|Ss|tT|Tt|uU|Uu|vV|Vv|wW|Ww|xX|Xx|yY|Yy|zZ|Zz')
    
def replace_polymer(text):
    return re.sub(p, '', text)

def count_length(text):
    new_text = replace_polymer(text)
    while new_text != text:
        text = new_text
        new_text = replace_polymer(text)
        
    return len(text)

def day5(text):
    alls = []
    for x in 'qwertyuiopasdfghjklzxcvbnm':
        t = text.replace(x, '').replace(x.upper(), '')
        alls.append((count_length(t), x))
    return min(alls)

day5(InputString(5))

(6550, 'x')

Under pressure, I can't think of an elegant method. Fortunately, this regex replacement is not too slow. I know I shouldn't use string replacement. Now, more relaxed, because we don't need the string, just the final length, or the number of remove operations.

Hmm, it's not easy as I think because when a match is removed, the cursor needs to go back one. A stack data structure is more suitable.

In [95]:
def match(a, b):
    return a != b and a.lower() == b.lower()

def count_length(text):
    stack = []
    for s in text:
        if stack and match(stack[-1], s):
            stack.pop()
        else:
            stack.append(s)
    return len(stack)

def day5x(text):
    return count_length(text)

day5x(InputString(5))

9172

## Day 6: Chronal Coordinates
[Problem Description](https://adventofcode.com/2018/day/6)

Well, the most tricky day so far. For part 1, I had to submit at least 5 times. I didn't write code to detect infinite regions, instead, relying on scanning through the output. Finite regions are the ones that don't change when you extend the boundary.

In [4]:
def mht(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

In [42]:
def exclude(a, min_x, max_x, min_y, max_y):
    return [p for p in a if p[0][0] > min_x and p[0][0] < max_x and p[0][1] > min_y and p[0][1] < max_y]

def day6(rows):
    coords = []
    lookup = defaultdict(int)
    for row in rows:
        p = tuple(map(int, row.split(',')))
        coords.append(p)
    
    min_x = min(coords, key=lambda x:x[0])[0]
    max_x = max(coords, key=lambda x:x[0])[0]
    min_y = min(coords, key=lambda x:x[1])[1]
    max_y = max(coords, key=lambda x:x[1])[1]
    print(min_x, max_x, min_y, max_y)
    for i in range(min_x-200, max_x+400):
        for j in range(min_y-200, max_y+400):
            distances = [(c, mht(c, (i, j))) for c in coords]
#             print(distances)
            min_distance = min(distances, key=lambda x: x[1])
#             print(min_distance)
            count = 0
            p = None
            for d in distances:
                if d[1] == min_distance[1]:
                    p = d[0]
                    count += 1
            if count == 1:
                lookup[p] += 1
                

    sorted_by_value = sorted(lookup.items(), key=lambda kv: kv[1], reverse=True)
#     print(sorted_by_value)
    return exclude(sorted_by_value, min_x, max_x, min_y, max_y)

In [41]:
day6(InputRows(6))

41 353 43 358


[((348, 53), 43485),
 ((78, 356), 35900),
 ((67, 53), 25000),
 ((304, 190), 16918),
 ((106, 338), 16239),
 ((224, 339), 15627),
 ((339, 256), 14582),
 ((202, 86), 12220),
 ((64, 294), 11700),
 ((65, 203), 9759),
 ((61, 123), 6254),
 ((166, 169), 5532),
 ((159, 270), 5472),
 ((336, 303), 4990),
 ((336, 283), 4942),
 ((328, 72), 4814),
 ((91, 234), 4277),
 ((277, 110), 3903),
 ((95, 85), 3743),
 ((236, 337), 3371),
 ((114, 146), 3118),
 ((234, 119), 2797),
 ((236, 191), 2686),
 ((229, 286), 2236),
 ((114, 260), 1789),
 ((115, 256), 1772),
 ((310, 234), 1641),
 ((242, 164), 1533),
 ((226, 298), 1526),
 ((225, 179), 1490),
 ((281, 232), 1485),
 ((263, 187), 1462),
 ((285, 279), 1414),
 ((82, 142), 1294),
 ((244, 318), 1284),
 ((310, 295), 1028),
 ((301, 335), 981),
 ((261, 267), 822),
 ((321, 335), 693),
 ((329, 305), 418),
 ((100, 143), 406),
 ((266, 253), 406),
 ((97, 140), 320),
 ((261, 258), 286),
 ((312, 341), 276)]

In [43]:
day6(InputRows(6))

41 353 43 358


[((348, 53), 126085),
 ((78, 356), 110400),
 ((67, 53), 67800),
 ((304, 190), 29318),
 ((106, 338), 28839),
 ((224, 339), 28827),
 ((339, 256), 27582),
 ((64, 294), 19800),
 ((202, 86), 18620),
 ((65, 203), 16359),
 ((61, 123), 10754),
 ((336, 303), 9390),
 ((336, 283), 9142),
 ((91, 234), 6177),
 ((328, 72), 6114),
 ((236, 337), 5971),
 ((166, 169), 5532),
 ((159, 270), 5472),
 ((277, 110), 3903),
 ((95, 85), 3743),
 ((114, 146), 3118),
 ((234, 119), 2797),
 ((236, 191), 2686),
 ((229, 286), 2236),
 ((114, 260), 1789),
 ((115, 256), 1772),
 ((310, 234), 1641),
 ((242, 164), 1533),
 ((226, 298), 1526),
 ((225, 179), 1490),
 ((281, 232), 1485),
 ((263, 187), 1462),
 ((285, 279), 1414),
 ((82, 142), 1294),
 ((244, 318), 1284),
 ((310, 295), 1028),
 ((301, 335), 981),
 ((261, 267), 822),
 ((321, 335), 693),
 ((329, 305), 418),
 ((100, 143), 406),
 ((266, 253), 406),
 ((97, 140), 320),
 ((261, 258), 286),
 ((312, 341), 276)]

Part 2 sucks. Very easy but rejected by the site. And it looks like there's a bug there.

In [62]:
def day6(rows, limit):
    coords = []
    lookup = defaultdict(int)
    for row in rows:
        p = tuple(map(int, row.split(',')))
        coords.append(p)
    
    min_x = min(coords, key=lambda x:x[0])[0]
    max_x = max(coords, key=lambda x:x[0])[0]
    min_y = min(coords, key=lambda x:x[1])[1]
    max_y = max(coords, key=lambda x:x[1])[1]

    print(min_x, max_x, min_y, max_y)
    count = 0
    for i in range(min_x-100, max_x+100):
        for j in range(min_y-100, max_y+100):
            distance = sum(mht(c, (i, j)) for c in coords)
            if distance < limit:
                count += 1
    return count

day6(InputRows(6), 10000)

41 353 43 358


36216

## Day 7: The Sum of Its Parts
[Problem Description](https://adventofcode.com/2018/day/7)

In [150]:
def day7a(rows):
    g = defaultdict(list)
    parent = defaultdict(list)
    for r in rows:
        o, d = r[5], r[36]
        g[o].append(d)
        parent[d].append(o)
        
    left = sorted(set(list(g.keys()) + list(flatten(g.values()))))
    avails = set(c for c in left if not parent[c])
#     print(avails)
    path = []
    while True:
        for n in left:
            if n in avails:
                path.append(n)
                left.remove(n)
                for c in g[n]:
                    if all(p in path for p in parent[c]):
                        avails.add(c)
                if not left:
                    return ''.join(path)
#                 print('add ', n)
#                 print('avails', avails)
                break
    
day7a(InputRows(7))

'JNOIKSYABEQRUVWXGTZFDMHLPC'

In [151]:
alpha = sorted('QWERTYUIOPASDFGHJKLZXCVBNM')

def day7b(rows):
    g = defaultdict(list)
    parent = defaultdict(list)
    for r in rows:
        o, d = r[5], r[36]
        g[o].append(d)
        parent[d].append(o)
        
    left = set(sorted(set(list(g.keys()) + list(flatten(g.values())))))
    avails = set(c for c in left if not parent[c])
    started = set()
    workers = [0, 0, 0, 0, 0]    
    count = 0
    running = dict()

    # Is any task available?
    while left:
        not_started = avails - started
#         print(not_started)
        if not_started:
            for m in sorted(not_started):
                # Is any worker available?
                for i in range(len(workers)):
                    # Yes, assign task with corresponding time
                    if not workers[i]:
                        workers[i] = running[m] = alpha.index(m) + 61
                        started.add(m)
                        break # Just need to assign once
        
#         print(count, running, workers, not_started)
        
        # Time ticking
        for i in range(len(workers)):
            if workers[i]:
                workers[i] -= 1
        to_delete = []
        for m in running:
            running[m] -= 1
            # A task has finished?
            if running[m] == 0:
                to_delete.append(m)
                left.remove(m)
                
                # Check more available tasks: they can only come from its children
                for c in g[m]:
                    if all(p not in left for p in parent[c]):
                        avails.add(c)

        for t in to_delete:
            del running[t]

        count += 1

    return count
    
day7b(InputRows(7))

1099

In [153]:
def day7a2(rows):
    g = defaultdict(list)
    parent = defaultdict(list)
    for r in rows:
        o, d = r[5], r[36]
        g[o].append(d)
        parent[d].append(o)
        
    left = set(sorted(set(list(g.keys()) + list(flatten(g.values())))))
    avails = set(c for c in left if not parent[c])
    path = []

    while left:
        not_started = avails - set(path)
        if not_started:
            m = min(not_started)
            path.append(m)
            left.remove(m)
                
            # Check more available tasks: they can only come from its children
            for c in g[m]:
                if all(p not in left for p in parent[c]):
                    avails.add(c)

    return ''.join(path)
    
day7a2(InputRows(7))
print(day7a2(InputRows(7)) == day7a(InputRows(7)))

True
